In [33]:
import os
from pathlib import Path

with open("/cs/student/msc/dsml/2023/mdavudov/NLP/Project_CodeNet/problem_descriptions/p02917.html") as f:
    data = f.read()
sample = "../Project_CodeNet/data/p00000/C++/s284212099.cpp"
problem_id = sample.strip('\n').split("/")[-3]

pd_path = f"../Project_CodeNet/problem_descriptions/{problem_id}.html"
print(pd_path)
# print(data)

p00000
../Project_CodeNet/problem_descriptions/p00000.html


In [24]:
from bs4 import BeautifulSoup
import re
soup = BeautifulSoup(data, 'html.parser')

# Find all headings and insert newlines if necessary
for heading in soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6']):
    if '\n' not in heading.next_sibling:  # Check if next element already has a newline
        heading.insert_after(BeautifulSoup('\n', 'html.parser')) 

text = soup.get_text()

# remove multiple newlines and just have one
#text = text.replace("\n\n", "\n")
text = re.sub(r'\n{1,}', '\n', text)

symbol_replacements = {
    r'\\leq': '≤', 
    r'\\geq': '≥', 
    r'\\max': 'max',
    r'\\min': 'min',
    # ... 
}

for pattern, replacement in symbol_replacements.items():
    text = re.sub(pattern, replacement, text)


print(text)


Score : 300 points
Problem Statement
There is an integer sequence A of length N whose values are unknown.
Given is an integer sequence B of length N-1 which is known to satisfy the following:
 B_i ≥ max(A_i, A_{i+1}) 
Find the maximum possible sum of the elements of A.
Constraints
All values in input are integers.
2 ≤ N ≤ 100
0 ≤ B_i ≤ 10^5
Input
Input is given from Standard Input in the following format:
N
B_1 B_2 ... B_{N-1}
Output
Print the maximum possible sum of the elements of A.
Sample Input 1
3
2 5
Sample Output 1
9
A can be, for example, ( 2 , 1 , 5 ), ( -1 , -2 , -3 ), or ( 2 , 2 , 5 ). Among those candidates, A = ( 2 , 2 , 5 ) has the maximum possible sum.
Sample Input 2
2
3
Sample Output 2
6
Sample Input 3
6
0 153 10 10 23
Sample Output 3
53



# Add promp template code here

In [2]:
# from prompt_utils import make_prompt_template, parse_code, cut_text, make_prompt_template_pd, parse_pd_html, make_simple_prompt_template
from prompt_utils import parse_pd_html
from pathlib import Path
from prompt_utils import gemmaRegularPromptTemplate, baseInstructionTemplate, baseExampleTemplate, baseCodeTaskDescription, incompleteCodeTemplate

#make instruction
# instruction = f"{baseInstructionTemplate} \n {baseExampleTemplate} \n {baseCodeTaskDescription(codeDescriptionString)}"
# gemmaRegularPromptTemplate(instruction = instruction,  response="", code=incompleteCodeTemplate(codeString))

def cut_text(txt: str, cut_ratio=0.1):
    '''
    Cut cut_ratio of the text from the end 
    '''
    stripped = txt.strip('\n ')
    end_idx = max(int(cut_ratio * len(stripped)), 1)
    return stripped[:-end_idx]

def get_prompt_templates(samples_path: Path, include_pd: bool = False, include_example: bool = False):
    prompt_templates = []

    with open(samples_path) as samples_file:
        samples = samples_file.readlines()
        for sample in samples:

            with open(sample.strip('\n')) as f:
                text = f.read()
                code_sample = cut_text(text)

            
            instruction = f"{baseInstructionTemplate()}"

            if include_example:
                instruction += f"\n{baseExampleTemplate()}"
                # template = make_prompt_template(code_sample)
            if(include_pd):
                problem_id = sample.strip('\n').split("/")[-3]
                pd_path = f"../Project_CodeNet/problem_descriptions/{problem_id}.html"
                pd = parse_pd_html(pd_path)
                instruction += f"\n{baseCodeTaskDescription(pd)}"
                # template = make_simple_prompt_template(code_sample, pd)


            template = gemmaRegularPromptTemplate(instruction = instruction,  response="", code=incompleteCodeTemplate(code_sample))

            prompt_templates.append(template)
            
    return prompt_templates  

prompt_templates = get_prompt_templates('./samples_small.txt', include_pd=True, include_example=True) #all

# prompt_templates = get_prompt_templates('./samples_small.txt', include_pd=False, include_example=True)#include example

# prompt_templates = get_prompt_templates('./samples_small.txt', include_pd=False, include_example=False)# no example no pd

print(prompt_templates[2])

[Instruction]
You are an assistant that helps users with writing compiler-friendly C++ programmes. Your outputs should be exclusively C++ programmes that can be compiled with C++17. 
Complete the code given in the input to produce a valid C++ program. The code to complete will be delimited by [C++] and [/C++]. Your response will be delimited with [Response] and [/Response].


[Example]

[C++]
#include <iostream>

using namespace std; 

int main() {
    cout << "Hello, 
[/C++]

[Response]
#include <iostream>

using namespace std; 

int main() {
    cout << "Hello, World! << endl;
}
[/Response]

[/Example]

[Code Problem Description]

Digit Number
Write a program which computes the digit number of sum of two integers a and b.
Input
There are several test cases. Each test case consists of two non-negative integers a and b which are separeted by a space in a line. The input terminates with EOF.
Constraints
0 ≤ a, b ≤ 1,000,000
The number of datasets ≤ 200
Output
Print the number of digits 

In [3]:
%set_env HF_HOME=/cs/student/projects3/COMP0087/grp2/hf-hub
%set_env TIKTOKEN_CACHE_DIR=/cs/student/projects3/COMP0087/grp2/tiktoken-cache

import os
#print evn vars
print(os.environ['HF_HOME'])
print(os.environ['TIKTOKEN_CACHE_DIR'])

from dataclasses import dataclass, field
from typing import Optional
from pathlib import Path
import json 
import numpy as np

from experiments import Experiment
from prompt_utils import make_prompt_template, parse_code, cut_text, make_prompt_template_pd, parse_pd_html, make_simple_prompt_template
from compiler_utils import try_compile_cpp
import os
import sys

argument = "gemma-7b-bnb-4bit"
exp_name = 6
@dataclass
class Evaluation:
    experiment: Experiment 
    evaluation_id: str 
    prompt: str 
    output: str 
    code_output: Optional[str] = field(default=None)

    @property
    def eval_folder(self) -> Path:
        return self.experiment.root_folder.joinpath(f'./evals/{self.evaluation_id}')

    @property 
    def prompt_file_path(self) -> Path:
        return self.eval_folder.joinpath('prompt.txt')

    @property 
    def output_file_path(self) -> Path:
        return self.eval_folder.joinpath('output.txt')

    @property 
    def code_output_file_path(self) -> Path:
        return self.eval_folder.joinpath('code_output.cpp')

    def __post_init__(self):
        self.eval_folder.mkdir(parents=True, exist_ok=True)

        self.code_output = parse_code(self.output)
        self.output = self.prompt + self.output

        to_write = [
            (self.prompt_file_path, self.prompt),
            (self.output_file_path, self.output),
            (self.code_output_file_path, self.code_output)
        ]
        for file_path, text in to_write:
            with open(file_path, 'w') as file:
                if(text is None):
                    text = "None"
                file.write(text)


prompt_templates = get_prompt_templates('./samples_small.txt', True)

experiment = Experiment(
    model=f"unsloth/{argument}",
    exp_type='eval',
    seq_length=8192,
    max_new_tokens=1024,
    exp_name=f'{argument}_ansible_test_{exp_name}'
) 

model, tokenizer = experiment.get_unsloth_model()




# model.config.output_hidden_states = True # for contrastive search

# print(model.config.output_hidden_states)

#prompts = [
#     tokenizer.apply_chat_template(template, tokenize=False)
#     for template in prompt_templates 
#]
prompts = prompt_templates

# prompts = [prompt + tokenizer.eos_token for prompt in prompts]#!!!!!!!!!!!!!!!!!!!!!!!!!! add eos token to end

sample_results = dict()

# If this is too big we OOM for some reason...
PROMPT_BATCH_SIZE = 2#5
PROMPT_BATCHES = (len(prompts) + PROMPT_BATCH_SIZE - 1) // PROMPT_BATCH_SIZE

#list of responses
responses = []

print("STARTING Inferance")
for prompt_batch_idx in range(PROMPT_BATCHES):
    prompt_batch = prompts[prompt_batch_idx * PROMPT_BATCH_SIZE: (prompt_batch_idx + 1) * PROMPT_BATCH_SIZE]
    inputs = tokenizer(prompt_batch, return_tensors = "pt", padding=True, truncation=True, max_length=8192).to('cuda')
    input_ids = inputs["input_ids"]

    outputs = model.generate(**inputs, 
                            max_new_tokens=100,#1024, 
                            pad_token_id=tokenizer.eos_token_id,

                            # use_cache = True,

                            #reduce chance for low ranked tokens
                            #  do_sample=True,
                            #  top_k=50,
                            
                            #lower temp -> more random - better for writing text
                            # do_sample=True,
                            # top_k=0, #deactive
                            # temperature=0.6,

                            # more dynamic, very heavy on Memory
                            #  num_beams=5,#5,
                            #  no_repeat_ngram_size=2,
                            #  num_return_sequences=5,#5,
                            #  early_stopping=True,

                            #Top-p (nucleus) sampling
                            do_sample=True,
                            top_p=0.92,
                            top_k=0,

                            
                            # papers:
                            #Exploring Parameter-Efficient Fine-Tuning Techniques for Code Generation with Large Language Models
                            #https://github.com/martin-wey/peft-llm-code/blob/main/test.py#L127
                            # num_beams=10,
                            # num_return_sequences=10,
                            # max_new_tokens=128#64 # args.max_target_length,
                            # stopping_criteria=StoppingCriteriaList(
                                # [EndOfFunctionCriteria(sample["input_ids"].shape[1], eof_string, tokenizer)])

                            #contrastive Search
                            #https://github.com/yxuansu/Contrastive_Search_Is_What_You_Need/blob/main/code_generation/inference.py
                            #Needs: model.config.output_hidden_states = True
                            #However this doesnt seem to be respected by the model
                            # penalty_alpha=0.4, 
                            # top_k=3,
                            
                            #PPOCoder
                            # do_sample=True,
                            # top_p=1.0,
                            # top_k=2,  

                            #Codex
                            # do_sample=True,
                            # top_p=0.95,
                            # top_k=0,                            

                            #StepCoder - encoding
                            # do_sample=True,
                            # temperature=0.8,
                            # top_p=0.9,
                            # top_k=0
                            #StepCoder - decoding
                            # do_sample=True,
                            # temperature=0.2,
                            # top_p=0.95,
                            # top_k=0
                             )

    for seq_idx, (prompt, output_sequence) in enumerate(zip(prompts, outputs)):
        
        #output = tokenizer.decode(output_sequence, skip_special_tokens=True)
        new_tokens = output_sequence[input_ids.shape[-1]:]

        numpy_array = new_tokens.cpu().numpy()  

        # Set NumPy's print options
        np.set_printoptions(threshold=np.inf)

        # print(numpy_array) 
        # print(new_tokens)
        output = tokenizer.decode(new_tokens, skip_special_tokens=False)


        responses.append({"output": output, "numpy_array":numpy_array, "new_tokens":new_tokens, "output_sequence":output_sequence, "prompt":prompt})

        
        # print(f"##### {seq_idx} begin #####")
        # print(output)
        # print(f"##### {seq_idx} end #####")
        # seq_id = prompt_batch_idx * PROMPT_BATCH_SIZE + seq_idx 
        # evaluation = Evaluation(experiment, seq_id, prompt, output)
        # compile_result = try_compile_cpp(src_path=evaluation.code_output_file_path)
        # sample_status = "Good" if compile_result.returncode == 0 else "Bad"
        # print(f'Sample {seq_id} is {sample_status}')
        # sample_results[seq_id] = {
        #     'Status': sample_status,
        #     'stderr': compile_result.stderr.decode('utf-8')
        # }

    # with open(experiment.root_folder.joinpath('results.json'), 'a') as results_file:
    #     json.dump(sample_results, results_file)
        
print("END Inferance")

env: HF_HOME=/cs/student/projects3/COMP0087/grp2/hf-hub
env: TIKTOKEN_CACHE_DIR=/cs/student/projects3/COMP0087/grp2/tiktoken-cache
/cs/student/projects3/COMP0087/grp2/hf-hub
/cs/student/projects3/COMP0087/grp2/tiktoken-cache
Experiment gemma-7b-bnb-4bit_ansible_test_6 already exists. Please choose an appropriate action:
D - delete previous and overwrite. I - ignore. A - Abort


HF_HOME: /cs/student/projects3/COMP0087/grp2/hf-hub
==((====))==  Unsloth: Fast Gemma patching release 2024.3
   \\   /|    GPU: NVIDIA GeForce RTX 3090 Ti. Max memory: 23.677 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.0. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.24. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


/cs/student/msc/dsml/2023/gbillenn/NLP/miniconda3/envs/md_unsloth/lib/python3.10/site-packages/transformers/quantizers/auto.py:155: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


Activating faster inference with unsloth
STARTING Inferance
END Inferance


In [4]:
#print the length of each of the responses
print("Length of each of the responses:")
for response in responses:
    print(len(response['output']))

# for each response, print the output
for response in responses:
    string = f"\n----------------------------------------------- \n{response['output']} \n"
    print(string)

Length of each of the responses:
500
453
500
292
83

----------------------------------------------- 
<bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos> 


----------------------------------------------- 
	cout<<h[9]<<endl;
	cout<<h[<bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><bos><b